In [2]:
import json
from watson_developer_cloud import AlchemyLanguageV1
keyList=['cb738b85c0e2d0094894bcfe8c73d12d73543c35','af89b7da4bab4becaad685eaff5e2b90','b4632297-1c70-4de2-821b-95d2c3245d01','49662623-dd08-4f6e-871b-af1dec4cdaa1','a249944a-48e3-44cc-b481-248f7fa1e2d7']
keyUse=[True,False,False,False,False]
currentKey=0;
maxKey=5
#api_key=keyList[0]
alchemy_language = AlchemyLanguageV1(api_key='cb738b85c0e2d0094894bcfe8c73d12d73543c35')
from urllib.request import urlopen
import re

In [3]:
def targeted_document_analysis(url_input, target_words):
    if (alchemy_language.targeted_sentiment(url=url_input, targets=target_words)["results"][0]['sentiment']['type'])=='neutral':
        return 0.0
    else:
        return (alchemy_language.targeted_sentiment(url=url_input, targets=target_words)["results"][0]['sentiment']['score'])
    pass

In [4]:
def document_sentiment(url_input):
    if (alchemy_language.sentiment(url=url_input)['docSentiment']['score'])=='neutral':
        return 0.0
    else:
        return(alchemy_language.sentiment(url=url_input)['docSentiment']['score'])

In [5]:
def targeted_sentence_analysis(text_input, target_words):
    if (alchemy_language.targeted_sentiment(text=text_input, targets=target_words) ["results"][0]['sentiment']['type'])=='neutral':
        return 0.0
    else:
        return(alchemy_language.targeted_sentiment(text=text_input, targets=target_words) ["results"][0]['sentiment']['score'])
    pass

In [6]:
def sentence_sentiment(text_input):
    
    if (alchemy_language.sentiment(text=text_input)['docSentiment']['type'])=='neutral':
        return 0.0
    else:
        return(alchemy_language.sentiment(text=text_input)['docSentiment']['score'])
    

In [7]:
def listOfSentences (url,word):
    HTMl = urllib.request.urlopen(url).read()
    return re.findall(r"([^>=]*?word[^.<]*\.)",HTML)

In [8]:
def compare_documents(array):
    prev = document_sentiment(array[0])
    for i in range (0,len(array)):
        temp = document_sentiment(array[i])
        if temp*prev<0:
            break
    if temp*prev<0:
        return False
    else:
        return True

In [9]:
def compare_sentences(array):
    prev = float(sentence_sentiment(array[0]))
    for i in range (0,len(array)):
        temp = float(sentence_sentiment(array[i]))
        if temp+prev<0:
            break
    if temp+prev<0:
        return False
    else:
        return True

In [10]:
def emotion_analysis(urlInput):
    return alchemy_language.emotion(url=urlInput)['docEmotions']

In [11]:
def compare_similar_sentences(subsequence ):
    retArray=[]
    retArray.append([])
    retArray.append([])
    #print (retArray)
    retval=sentence_sentiment(subsequence[0])
    if (retval==None):
        return None
    prev = float(retval)
    for i in range (1,len(subsequence)):
        temp = float(sentence_sentiment(subsequence[i]))
        
        if temp*prev <= 0:
            #print("Flip Flopped")
            retArray[0].append(subsequence[i])
        else:
            #print("Didn't Flop Flop")
            retArray[1].append(subsequence[i])


    return retArray
#   Look through the stence array
#   put all the words in sentence into a dicitonary
#
#
#
def dumbWord(word):
    dumbWords= ["and", "or", "but", "nor", "so", "for", "yet", "after", "although", \
    "as", "as", "if", "as", "long", "as", "because", "before", "even", "if", "even", \
    "though", "once", "since", "so", "that", "though", "till", "unless", "until", "what", \
    "when", "whenever", "wherever", "whether", "while"]

    if (word.lower() in dumbWords):
        return True

    return False

THRESHOLD = 3
MIN_WORDS = 3
SIMILARITY = .5



def mostSimilarTo(sentence, sentenceArray):
    similarSentences = []
    sentenceDict = {}
    notDumbWords=0
    for word in sentence.split():
        #print (word)
        word = word.lower()
        if not dumbWord(word):
            notDumbWords += 1
            if word not in sentenceDict:
                sentenceDict[word]=1
            else:    
                sentenceDict[word] += 1
               
    

    for potentialSentence in sentenceArray:
        sameWords=0
        wordCount =0
        if potentialSentence != sentence:
            
            for word in potentialSentence.split() :
                word = word.lower()
                if not dumbWord(word):
                    wordCount+=1
                    if word  in sentenceDict:
                        sameWords+=1

       # print (sameWords)
       # print (wordCount)
        if(wordCount!=0):
            if sameWords >=wordCount*SIMILARITY:
                similarSentences.append(potentialSentence)

    return similarSentences


def flipFlopped(sentenceArray):
    retArr=[]
    for sentence in sentenceArray:
        #print (sentence)
        if len(sentence.split()) >= MIN_WORDS :
            
            retSentenceArray = mostSimilarTo(sentence, sentenceArray)
            print ("The current sentence is: "+sentence+" and it is most similar to: ",end="")
            print (retSentenceArray)
            print ("\n")
            if(  len(retSentenceArray) != 0):
              
                retSentenceArray.insert(0, sentence)
                print(retSentenceArray)
                retStatus = compare_similar_sentences(retSentenceArray)
                if (retStatus==None ):
                    return None
                retArr.append(sentence)
                retArr.append(retStatus)
            
    return retArr

In [125]:
flipFlopped(['I hate war war is bad war is not good','non no no yes hello hi there', 'These are not different and cool', 'these  are different and cool','I love war war is okay war is fun', 'War sounds good is good great war'])


#flipFlopped(['The sky is blue today', 'the sky is green today'])



#flipFlopped( ['Torture works', 'OK, folks', 'You know, I have these guys -- "Torture doesn\'t work', '-- believe me, it works', 'And waterboarding is your minor form', "Some people say it's not actually torture", "Let's assume it is", 'But they asked me the question, What do you think of waterboarding', 'Absolutely fine', 'But we should go much stronger than waterboarding', "That's the way I feel", "They're chopping off heads", "Believe me, we should go much stronger, because our country's in trouble", "We're in danger", 'We have people that want to do really bad things', 'Remember the power of weaponry', "This isn't 100 years ago where we fight hand to hand combat", 'This is weapons that are so destructive -- so destructive -- that the world could end', 'We have to be very strong, we have to be very vigilant, we have to be very tough', "Waterboarding is fine, but it's not nearly tough enough, OK", '\n\nOn the issue of torture, Mr', 'Trump suggested he had changed his mind about the value of waterboarding after talking with James N', 'Mattis, a retired Marine Corps general, who headed the United States Central Command', '\n\n“He said, ‘I’ve never found it to be useful,’” Mr', 'Trump said', 'He added that Mr', 'Mattis found more value in building trust and rewarding cooperation with terrorism suspects: “‘Give me a pack of cigarettes and a couple of beers, and I’ll do better', '’”\n\n“I was very impressed by that answer,” Mr', 'Trump said', '\n\nTorture, he said, is “not going to make the kind of a difference that a lot of people are thinking', '”'])





The current sentence is: I hate war war is bad war is not good and it is most similar to: ['I love war war is okay war is fun', 'War sounds good is good great war']


['I hate war war is bad war is not good', 'I love war war is okay war is fun', 'War sounds good is good great war']
The current sentence is: non no no yes hello hi there and it is most similar to: []


The current sentence is: These are not different and cool and it is most similar to: ['these  are different and cool']


['These are not different and cool', 'these  are different and cool']
The current sentence is: these  are different and cool and it is most similar to: ['These are not different and cool']


['these  are different and cool', 'These are not different and cool']
The current sentence is: I love war war is okay war is fun and it is most similar to: ['I hate war war is bad war is not good']


['I love war war is okay war is fun', 'I hate war war is bad war is not good']
The current sentence is: War sounds good

['I hate war war is bad war is not good',
 [['I love war war is okay war is fun', 'War sounds good is good great war'],
  []],
 'These are not different and cool',
 [['these  are different and cool'], []],
 'these  are different and cool',
 [['These are not different and cool'], []],
 'I love war war is okay war is fun',
 [['I hate war war is bad war is not good'], []],
 'War sounds good is good great war',
 [['I hate war war is bad war is not good'],
  ['I love war war is okay war is fun']]]

In [12]:
import re

def fileio( filename ):
    text= None
    with open(filename) as file:
        text=file.read()

    sentences = re.split(r' *[\.\?!][\'"\)\]]* *', text)

    print (sentences)
    retArray = flipFlopped(sentences)
    
    print (retArray)
    for sentence in retArray:
        print ("Original sentence:", end=" ")
        print (sentence)
        print ("List of Flipflops: ",end="") 
        print (sentence[0])
        print ("List of Consitent statements: ",end="") 
        print (sentence[1])
        print ('\n')

    
    


def stdio( filename ):
    return

    # open the file
    # put it into a list of setneces 

    # retArray = flipFlopped(sentenceArray)


    # print retArray

fileio("test2.txt")

['Torture works', 'OK, folks', 'You know, I have these guys -- "Torture doesn\'t work', '-- believe me, it works', 'And waterboarding is your minor form', "Some people say it's not actually torture", "Let's assume it is", 'But they asked me the question, What do you think of waterboarding', 'Absolutely fine', 'But we should go much stronger than waterboarding', "That's the way I feel", "They're chopping off heads", "Believe me, we should go much stronger, because our country's in trouble", "We're in danger", 'We have people that want to do really bad things', 'Remember the power of weaponry', "This isn't 100 years ago where we fight hand to hand combat", 'This is weapons that are so destructive -- so destructive -- that the world could end', 'We have to be very strong, we have to be very vigilant, we have to be very tough', "Waterboarding is fine, but it's not nearly tough enough, OK", '\n\nOn the issue of torture, Mr', 'Trump suggested he had changed his mind about the value of waterb

In [ ]:
except:
        found=False
        newKey=-1
        for i in range(currentKey,maxKey):
            if(keyUse[i] == False):
                found=True
                keyUse[i]=True
                newKey=i
                break
        
        if (not found) :
            print("Credentials, over used for the day, SORRY!")
            return None
        else:
            alchemy_language = AlchemyLanguageV1(api_key=keyList[newKey])
            sentence_sentiment(text_input)

In [32]:
import nltk
import gensim

In [1]:
trained_model.similarity('woman', 'man') 
0.73723527

NameError: name 'trained_model' is not defined